# Assignment 7

In [170]:
# libraries for assignment
import pandas as pd
import os
from pathlib import Path
import hashlib
import pygeohash

In [171]:
# establishing directories
current_dir = Path(os.getcwd()).absolute()
results_dir = current_dir.joinpath('results')
kv_dir = results_dir.joinpath("kv")
kv_dir.mkdir(parents=True, exist_ok=True)
hash_dir = results_dir.joinpath("hash")
hash_dir.mkdir(parents=True, exist_ok=True)

## 7.1

### 7.1.a

In [81]:
# import data
routes = pd.read_parquet("E:/Documents/GitHub/dsc650/dsc650/assignments/assignment03/results/routes.parquet")

In [82]:
routes

,airline,src_airport,dst_airport,codeshare,equipment
0,"{'active': True, 'airline_id': 410, 'alias': '...","{'airport_id': 2965.0, 'altitude': 89.0, 'city...","{'airport_id': 2990.0, 'altitude': 411.0, 'cit...",False,[CR2]
1,"{'active': True, 'airline_id': 410, 'alias': '...","{'airport_id': 2966.0, 'altitude': -65.0, 'cit...","{'airport_id': 2990.0, 'altitude': 411.0, 'cit...",False,[CR2]
2,"{'active': True, 'airline_id': 410, 'alias': '...","{'airport_id': 2966.0, 'altitude': -65.0, 'cit...","{'airport_id': 2962.0, 'altitude': 1054.0, 'ci...",False,[CR2]
3,"{'active': True, 'airline_id': 410, 'alias': '...","{'airport_id': 2968.0, 'altitude': 769.0, 'cit...","{'airport_id': 2990.0, 'altitude': 411.0, 'cit...",False,[CR2]
4,"{'active': True, 'airline_id': 410, 'alias': '...","{'airport_id': 2968.0, 'altitude': 769.0, 'cit...","{'airport_id': 4078.0, 'altitude': 365.0, 'cit...",False,[CR2]
...,...,...,...,...,...
67658,"{'active': True, 'airline_id': 4178, 'alias': ...","{'airport_id': 6334.0, 'altitude': 41.0, 'city...","{'airport_id': 3341.0, 'altitude': 20.0, 'city...",False,[SF3]
67659,"{'active': True, 'airline_id': 19016, 'alias':...","{'airport_id': 4029.0, 'altitude': 588.0, 'cit...","{'airport_id': 2912.0, 'altitude': 2058.0, 'ci...",False,[734]
67660,"{'active': True, 'airline_id': 19016, 'alias':...","{'airport_id': 2912.0, 'altitude': 2058.0, 'ci...","{'airport_id': 4029.0, 'altitude': 588.0, 'cit...",False,[734]
67661,"{'active': True, 'airline_id': 19016, 'alias':...","{'airport_id': 2912.0, 'altitude': 2058.0, 'ci...","{'airport_id': 2913.0, 'altitude': 2927.0, 'ci...",False,[734]


In [83]:
# list of partitions
partitions = (
        ('A', 'A'), ('B', 'B'), ('C', 'D'), ('E', 'F'),
        ('G', 'H'), ('I', 'J'), ('K', 'L'), ('M', 'M'),
        ('N', 'N'), ('O', 'P'), ('Q', 'R'), ('S', 'T'),
        ('U', 'U'), ('V', 'V'), ('W', 'X'), ('Y', 'Z')
    )

In [84]:
def key(route):
    """
    creates key concatenating src and dst airport codes
    """
    if route['src_airport'] is None or route['dst_airport'] is None:
        return " "
    else:
        return route["src_airport"]['iata'] + route['dst_airport']['iata']

In [85]:
def partition_key(key):
    """
    creates partition key derived from key
    """
    kv_key= " "
    if key is None:
        kv_key = ' '
    else:
        for partition in partitions:
            if key[0] == partition[0] or key[0] == partition[1]:
                kv_key = partition
    return kv_key

In [86]:
# create keys
routes['key'] = routes.apply(key, axis=1)

In [87]:
# create kv_keys
kv_key = routes['key'].apply(partition_key)

In [88]:
def partitions(kv_key):
    """
    converts kv_key into format desired for output
    """
    partitions = []
    for partition in kv_key:
        if len(partition) == 1:
            partitions.append(None)
        else:
            if len(set(partition)) == 1:
                partitions.append(partition[0])
            else:
                partitions.append (partition[0] + '-' + partition[1])
    return partitions

In [89]:
# create desired partition columns
partition_cols = partitions(kv_key)

In [90]:
# make into new column
routes['kv_key'] = partition_cols

In [92]:
# write to parquet
kv_path =   kv_dir.joinpath('kv.parquet')
routes.to_parquet(kv_path, partition_cols='kv_key')

### 7.1.b

In [93]:
def hash_key(key):
    m = hashlib.sha256()
    m.update(str(key).encode('utf-8'))
    return m.hexdigest()

In [94]:
# make hash of keys from 7.1.a
routes['hashed'] = routes['key'].apply(hash_key)

In [95]:
routes

,airline,src_airport,dst_airport,codeshare,equipment,key,kv_key,hashed
0,"{'active': True, 'airline_id': 410, 'alias': '...","{'airport_id': 2965.0, 'altitude': 89.0, 'city...","{'airport_id': 2990.0, 'altitude': 411.0, 'cit...",False,[CR2],AERKZN,A,300e091561e78892d58f3ab2832cb2011a8c560e335cd7...
1,"{'active': True, 'airline_id': 410, 'alias': '...","{'airport_id': 2966.0, 'altitude': -65.0, 'cit...","{'airport_id': 2990.0, 'altitude': 411.0, 'cit...",False,[CR2],ASFKZN,A,bef4f5e5d18146ae50da93b82c5a176860ba626f1bb63c...
2,"{'active': True, 'airline_id': 410, 'alias': '...","{'airport_id': 2966.0, 'altitude': -65.0, 'cit...","{'airport_id': 2962.0, 'altitude': 1054.0, 'ci...",False,[CR2],ASFMRV,A,4c5532465403f4892740d53aacedd1cf0290ba51966624...
3,"{'active': True, 'airline_id': 410, 'alias': '...","{'airport_id': 2968.0, 'altitude': 769.0, 'cit...","{'airport_id': 2990.0, 'altitude': 411.0, 'cit...",False,[CR2],CEKKZN,C-D,010aa3ec77dd01799a8d0aa18bb10e405e73c74e945505...
4,"{'active': True, 'airline_id': 410, 'alias': '...","{'airport_id': 2968.0, 'altitude': 769.0, 'cit...","{'airport_id': 4078.0, 'altitude': 365.0, 'cit...",False,[CR2],CEKOVB,C-D,b67af4a1b0947c107569c06a5ece40dcc771f6f137246d...
...,...,...,...,...,...,...,...,...
67658,"{'active': True, 'airline_id': 4178, 'alias': ...","{'airport_id': 6334.0, 'altitude': 41.0, 'city...","{'airport_id': 3341.0, 'altitude': 20.0, 'city...",False,[SF3],WYAADL,W-X,c9833bd0ea87896f10af539a03fdc95e57367206914fa6...
67659,"{'active': True, 'airline_id': 19016, 'alias':...","{'airport_id': 4029.0, 'altitude': 588.0, 'cit...","{'airport_id': 2912.0, 'altitude': 2058.0, 'ci...",False,[734],DMEFRU,C-D,b33af9767e5b69f5d6f113176c6de871c3d2368ecaf418...
67660,"{'active': True, 'airline_id': 19016, 'alias':...","{'airport_id': 2912.0, 'altitude': 2058.0, 'ci...","{'airport_id': 4029.0, 'altitude': 588.0, 'cit...",False,[734],FRUDME,E-F,8eeb948170942cfa940945b86d14df92e4115b16207b9d...
67661,"{'active': True, 'airline_id': 19016, 'alias':...","{'airport_id': 2912.0, 'altitude': 2058.0, 'ci...","{'airport_id': 2913.0, 'altitude': 2927.0, 'ci...",False,[734],FRUOSS,E-F,56977a878a991ac99cd62087a1860b048d140b69e24ce9...


In [114]:
def hash_key(hashed):
    """
    makes key of first symbol of hash key
    """
    partitions = []
    for h in hashed:
        partitions.extend(h[0].upper())
    return partitions

In [115]:
routes['hash_key'] = hash_key(routes['hashed'])

In [116]:
# outputs to results
hash_path = hash_dir.joinpath('hash.parquet')
routes.to_parquet(hash_path, partition_cols='hash_key')

### 7.1.c

In [158]:
# geohash airport location
routes['src_airport_geohash'] = routes['src_airport'].apply(
    lambda row: pygeohash.encode(row['latitude'], row['longitude']) if row is not None else None
)
def determine_location(src_airport_geohash):
    """
    takes geohash and determines closest data center using haversine distance function
    """
    locations = dict(
        central= pygeohash.encode(41.1544433, -96.0422378),
        west = pygeohash.encode(45.5945645,  -121.1786823),
        east = pygeohash.encode(39.08344,  -77.6497145)
    )
    if src_airport_geohash is None:
        return None
    else:
        distances = [['central' ,pygeohash.geohash_haversine_distance(src_airport_geohash, locations['central'])],
                     ['west', pygeohash.geohash_haversine_distance(src_airport_geohash, locations['west'])],
                     ['east', pygeohash.geohash_haversine_distance(src_airport_geohash, locations['east'])]] #TODO: a list of centers and distances using the pygeohash.geohash_haversine_distance function
    # sorts distances
    dist_sort = sorted(distances, key=lambda x:x[1])
    return dist_sort[0][0]

routes['location'] = routes['src_airport_geohash'].apply(determine_location)
routes.to_parquet('results/geo', partition_cols=['location'])

### 7.1.d

In [162]:
def balance_partitions(keys, num_partitions):
    """
     takes as input a list of keys and the number of partitions 
     returns a list of keys sorted into the specified number of partitions
    """
    partitions = []
    sorted_keys = sorted(keys)
    num_keys = len(sorted_keys)
    partition_size = num_keys // num_partitions
    
    for i in range(num_partitions):
        start = i * partition_size
        end = (i + 1) * partition_size
        partition = sorted_keys[start:end]
        
        if i == num_partitions - 1:
            partition = sorted_keys[start:]
        partitions.append(partition)
        
    return partitions

In [168]:
keys = ['A1', 'B2', 'C3', 'D4', 'E5', 'F6', 'G7', 'H8', 
        'I9', 'J10', 'K11', 'L12', 'M13', 'N14', 'O15', 
        'P16', 'Q17', 'R18', 'S19', 'T20', 'U21', 'V22', 
        'W23', 'X24', 'Y25', 'Z26', 'AA27', 'BB28', 'CC29', 'DD30']

In [169]:
balance_partitions(keys, 5)

[['A1', 'AA27', 'B2', 'BB28', 'C3', 'CC29'],
 ['D4', 'DD30', 'E5', 'F6', 'G7', 'H8'],
 ['I9', 'J10', 'K11', 'L12', 'M13', 'N14'],
 ['O15', 'P16', 'Q17', 'R18', 'S19', 'T20'],
 ['U21', 'V22', 'W23', 'X24', 'Y25', 'Z26']]